# Sage worksheet for "The asymptotic Mahler measure of Gaussian periods"

This worksheet accompanies the paper *The asymptotic Mahler measure of Gaussian periods* by Gunther Cornelissen, David Hokken, and Berend Ringeling.

### Preambule

In [ ]:
import itertools
import math

R.<x> = PolynomialRing(QQ)
Z.<X> = PolynomialRing(ZZ) 
CBF = ComplexBallField(50)

def Gal(f):
    # Given an irreducible polynomial f of sufficiently small degree, returns its Galois group over Q
    return NumberField(f,'a').galois_group(type="pari")

def f_rev(f):
    # Given a polynomial f, returns the reversal f_rev of f
    T = var('T')
    deg = f.degree()
    v = ((T^deg*f(1/T)).expand()).list()
    return R(v)

def realmahler(f):
    # Define the Mahler measure of f, here in a slightly altered form. The function
    # realmahler(f) returns the 'true' Mahler measure of f only if f is squarefree
    # and all roots of f are real. Otherwise it returns 0
    roots = f.roots(ring=RR)
    if len(roots) != f.degree():
        return 0
    else:
        return prod([abs(rt[0]) for rt in roots if abs(rt[0]) > 1])

### Degree 7

The goal is to show that the minimal Mahler measure of a degree-7 irreducible polynomial with cyclic Galois group is attained by the minimal polynomial of the following algebraic integer:

In [ ]:
zeta = QQbar(exp(2*pi*I/29))
a = zeta + zeta.conjugate() + zeta^17 + zeta.conjugate()^17
f = a.minpoly()
f, realmahler(f)

In [ ]:
## THE ALGORITHM

# First, give an odd prime n and an upper bound B for the smallest Mahler measure in degree n
n = 7
B = 24.216570147

# Create list of conductors
condlist = []
for m in xsrange(1,int(1+B^(2*n/(n-1)))):
    for (prime, prime_valuation) in factor(m):
        if prime%n not in [0,1]:
            break
        if prime%n == 0 and prime_valuation != 2:
            break
        if prime%n == 1 and prime_valuation != 1:
            break
        if (prime, prime_valuation) == factor(m)[-1]:
            # i.e., if m satisfies Lemma 12.1, that is, each prime divisor p of m satisfies
            # (1) p = 0 or 1 mod n, and
            # (2) if p = 0 mod n then m is divisible by p exactly twice, and
            # (3) if p = 1 mod n then p^2 does not divide m,
            # then m is a feasible conductor, so we should append m to the conductor list.
            condlist.append(m)

# For n=7, B=24.216570147, this gives the following list of conductors:
# [29, 43, 49, 71, 113, 127, 197, 211, 239, 281, 337, 343, 379, 421, 449, 463, 491, 547, 617, 631, 659,
# 673, 701, 743, 757, 827, 883, 911, 953, 967, 1009, 1051, 1093, 1163, 1247, 1289, 1303, 1373, 1421, 1429,
# 1471, 1499, 1583, 1597, 1667]
# All these are prime, except the following ones:
# [49, 1247, 1421] = [7^2, 29*43, 7^2*29]

# Start of the algorithm. Here, we loop over all conductors. In practice, it is computationally infeasible
# to run this algorithm for all elements of condlist -- the value 7^2 will take many years to terminate.
# However, running this for all other conductors is doable in about a week on a standard laptop.
for m in condlist: 
    # STEP (i) 
    # construct the set S. First, generate all polynomials over
    # the finite fields F_p where p varies over the prime divisors 
    # of m; call m1 the product of these p.
    pollistpre = []
    m1 = 1
    for fac in factor(m):
        p = fac[0]
        m1 *= p # construct m1, the radical of m
        FF.<y> = PolynomialRing(GF(p))
        pollistpre_entry = []
        for a in range(p):
            if 0 <= a^n % p < B:
                pollistpre_entry.append((p, FF((y+a)^n)))
        pollistpre.append(pollistpre_entry)
    
    # Apply CRT to obtain S (here as the obvious lifts to polynomials over Z)
    pollist = []
    for v in itertools.product(*pollistpre):
        moduli = [Z(v[i][0]) for i in range(len(v))]
        pols = [Z(v[i][1]) for i in range(len(v))]
        crtpol = CRT(pols, moduli)
        # Like before, check if constant coefficient of crtpol is sufficiently small
        if int(mod(crtpol(0), prod(moduli))) < B:
            # If so, reduce all coefficients of h to lie between 0 and prod(moduli) = m1.
            # This step is necessary because the CRT command doesn't do this
            # and we need this for the coefficient bounds below
            h = Z([mod(coeff, prod(moduli)) for coeff in crtpol.list()])
            # STEP (ii)
            # append h to pollist, but only if reversal of h is not in pollist
            if not Z(f_rev(h)) in pollist:
                pollist.append(h)
    
    # STEP (iii)
    # Calculate maximal coefficient ranges to form other polynomials over Z
    # from the reduced ones generated above in pollist, that actually live
    # over Z/m1Z
    coefranges = []
    # Recall that the constant coefficient is positive, whereas the others 
    # might be negative
    coefranges.append(range(1+int(B/m1)))
    for j in range(1, n):
        coefranges.append(range(int(-1+(1-binomial(n,j)*B)/m1), 1+int((binomial(n,j)*B)/m1)))
        
    pollist = list(reversed(pollist)) 
    
    print('')
    print(m, m1, len(pollist), coefranges)
    print('')
    
    ## Start of verification
    for h in pollist:
        print(h)
        for v0 in coefranges[0]:
            for v1 in coefranges[1]:
                # if h(0) vanishes, then any lift f of h to Z where the constant coefficient
                # remains 0 is reducible, and hence need not be considered.
                if h(0) == 0 and v0 == 0:
                    break
                for v2 in coefranges[2]:
                    for v3 in coefranges[3]:
                        for v4 in coefranges[4]:
                            for v5 in coefranges[5]:
                                for v6 in coefranges[6]:
                                    f = h + m1*(v6*X^6 + v5*X^5 + v4*X^4 + v3*X^3 + v2*X^2 + v1*X + v0)
                                    # STEP (iv)
                                    disc = f.disc()
                                    if disc.is_square() and 0 < abs(f(1)*f(-1))*disc < B^(2*n) and (m^(n-1)).divides(disc):
                                        print('')
                                        # STEP (v)
                                        if f.is_irreducible():
                                            # STEP (vi)
                                            M = realmahler(f)
                                            if M > 1:
                                                # STEP (vii)
                                                print(f)
                                                print(Gal(f))
                                                print(M)
                                                print(disc.factor())
                                                print('')

### Degree 5

We also check the easier degree-$5$ case:

In [ ]:
zeta = QQbar(exp(2*pi*I/11))
a = zeta + zeta.conjugate()
f = a.minpoly()
f, realmahler(f)

Here follows the implementation of the algorithm in degree 5.

In [ ]:
n = 5 # degree
B = 4.229 # upper bound for the minimal Mahler measure

# Create list of conductors
condlist = []
for m in xsrange(1,int(1+B^(2*n/(n-1)))):
    for (prime, prime_valuation) in factor(m):
        if prime%n not in [0,1]:
            break
        if prime%n == 0 and prime_valuation != 2:
            break
        if prime%n == 1 and prime_valuation != 1:
            break
        if (prime, prime_valuation) == factor(m)[-1]:
            # i.e., if m satisfies Lemma 12.1, that is, each prime divisor p of m satisfies
            # (1) p = 0 or 1 mod n, and
            # (2) if p = 0 mod n then m is divisible by p exactly twice, and
            # (3) if p = 1 mod n then p^2 does not divide m,
            # then m is a feasible conductor, so we should append m to the conductor list.
            condlist.append(m)


# Start of the algorithm   
for m in condlist:
    # STEP (i) 
    # construct the set S. First, generate all polynomials over
    # the finite fields F_p where p varies over the prime divisors 
    # of m; call m1 the product of these p.
    pollistpre = []
    m1 = 1
    for fac in factor(m):
        p = fac[0]
        m1 *= p # construct m1, the radical of m
        FF.<y> = PolynomialRing(GF(p))
        pollistpre_entry = []
        for a in range(p):
            if 0 <= a^n % p < B:
                pollistpre_entry.append((p, FF((y+a)^n)))
        pollistpre.append(pollistpre_entry)
    
    # Apply CRT to obtain S (here as the obvious lifts to polynomials over Z)
    pollist = []
    for v in itertools.product(*pollistpre):
        moduli = [Z(v[i][0]) for i in range(len(v))]
        pols = [Z(v[i][1]) for i in range(len(v))]
        crtpol = CRT(pols, moduli)
        # Like before, check if constant coefficient of crtpol is sufficiently small
        if int(mod(crtpol(0), prod(moduli))) < B:
            # If so, reduce all coefficients of h to lie between 0 and prod(moduli) = m1.
            # This step is necessary because CRT doesn't do this (against what you would
            # expect) and we need this for the coefficient bounds below
            h = Z([mod(coeff, prod(moduli)) for coeff in crtpol.list()])
            # STEP (ii)
            # append h to pollist, but only if reversal of h is not in pollist
            if not Z(f_rev(h)) in pollist:
                pollist.append(h)
    
    # STEP (iii)
    # Calculate maximal coefficient ranges to form other polynomials over Z
    # from the reduced ones generated above in pollist, that actually live
    # over Z/m1Z
    coefranges = []
    # Recall that the constant coefficient is positive, whereas the others 
    # might be negative
    coefranges.append(range(1+int(B/m1)))
    for j in range(1, n):
        coefranges.append(range(int(-1+(1-binomial(n,j)*B)/m1), 1+int((binomial(n,j)*B)/m1)))
        
    pollist = list(reversed(pollist)) 
    
    print('')
    print(m, m1, len(pollist), coefranges)
    print('')
    
    ## Start of verification
    for h in pollist:
        print(h)
        for v0 in coefranges[0]:
            for v1 in coefranges[1]:
                # if h(0) vanishes, then any lift f of h to Z where the constant coefficient
                # remains 0 is reducible, and hence need not be considered.
                if h(0) == 0 and v0 == 0:
                    break
                for v2 in coefranges[2]:
                    for v3 in coefranges[3]:
                        for v4 in coefranges[4]:
                            f = h + m1*(v4*X^4 + v3*X^3 + v2*X^2 + v1*X + v0)
                            # STEP (iv)
                            disc = f.disc()
                            if disc.is_square() and 0 < abs(f(1)*f(-1))*disc < B^(2*n) and (m^(n-1)).divides(disc):
                                print('')
                                print('test 1')
                                # STEP (v)
                                if f.is_irreducible():
                                    print('test 2')
                                    # STEP (vi)
                                    M = realmahler(f)
                                    print('test 3')
                                    if M > 1:
                                        # STEP (vii)
                                        print(f)
                                        print(Gal(f))
                                        print(M)
                                        print(disc.factor())
                                        print('')

### Degree 3

In [ ]:
zeta = QQbar(exp(2*pi*I/7))
a = zeta + zeta.conjugate()
f = a.minpoly()
f, realmahler(f)

In [ ]:
n = 3 # degree
B = 2.247 # upper bound for the minimal Mahler measure

# Create list of conductors
condlist = []
for m in xsrange(1,int(1+B^(2*n/(n-1)))):
    for (prime, prime_valuation) in factor(m):
        if prime%n not in [0,1]:
            break
        if prime%n == 0 and prime_valuation != 2:
            break
        if prime%n == 1 and prime_valuation != 1:
            break
        if (prime, prime_valuation) == factor(m)[-1]:
            # i.e., if m satisfies Lemma 12.1, that is, each prime divisor p of m satisfies
            # (1) p = 0 or 1 mod n, and
            # (2) if p = 0 mod n then m is divisible by p exactly twice, and
            # (3) if p = 1 mod n then p^2 does not divide m,
            # then m is a feasible conductor, so we should append m to the conductor list.
            condlist.append(m)


# Start of the algorithm   
for m in condlist:
    # STEP (i) 
    # construct the set S. First, generate all polynomials over
    # the finite fields F_p where p varies over the prime divisors 
    # of m; call m1 the product of these p.
    pollistpre = []
    m1 = 1
    for fac in factor(m):
        p = fac[0]
        m1 *= p # construct m1, the radical of m
        FF.<y> = PolynomialRing(GF(p))
        pollistpre_entry = []
        for a in range(p):
            if 0 <= a^n % p < B:
                pollistpre_entry.append((p, FF((y+a)^n)))
        pollistpre.append(pollistpre_entry)
    
    # Apply CRT to obtain S (here as the obvious lifts to polynomials over Z)
    pollist = []
    for v in itertools.product(*pollistpre):
        moduli = [Z(v[i][0]) for i in range(len(v))]
        pols = [Z(v[i][1]) for i in range(len(v))]
        crtpol = CRT(pols, moduli)
        # Like before, check if constant coefficient of crtpol is sufficiently small
        if int(mod(crtpol(0), prod(moduli))) < B:
            # If so, reduce all coefficients of h to lie between 0 and prod(moduli) = m1.
            # This step is necessary because CRT doesn't do this (against what you would
            # expect) and we need this for the coefficient bounds below
            h = Z([mod(coeff, prod(moduli)) for coeff in crtpol.list()])
            # STEP (ii)
            # append h to pollist, but only if reversal of h is not in pollist
            if not Z(f_rev(h)) in pollist:
                pollist.append(h)
    
    # STEP (iii)
    # Calculate maximal coefficient ranges to form other polynomials over Z
    # from the reduced ones generated above in pollist, that actually live
    # over Z/m1Z
    coefranges = []
    # Recall that the constant coefficient is positive, whereas the others 
    # might be negative
    coefranges.append(range(1+int(B/m1)))
    for j in range(1, n):
        coefranges.append(range(int(-1+(1-binomial(n,j)*B)/m1), 1+int((binomial(n,j)*B)/m1)))
        
    pollist = list(reversed(pollist)) 
    
    print('')
    print(m, m1, len(pollist), coefranges)
    print('')
    
    ## Start of verification
    for h in pollist:
        print(h)
        for v0 in coefranges[0]:
            for v1 in coefranges[1]:
                # if h(0) vanishes, then any lift f of h to Z where the constant coefficient
                # remains 0 is reducible, and hence need not be considered.
                if h(0) == 0 and v0 == 0:
                    break
                for v2 in coefranges[2]:
                    f = h + m1*(v2*X^2 + v1*X + v0)
                    # STEP (iv)
                    disc = f.disc()
                    if disc.is_square() and 0 < abs(f(1)*f(-1))*disc < B^(2*n) and (m^(n-1)).divides(disc):
                        print('')
                        print('test 1')
                        # STEP (v)
                        if f.is_irreducible():
                            print('test 2')
                            # STEP (vi)
                            M = realmahler(f)
                            print('test 3')
                            if M > 1:
                                # STEP (vii)
                                print(f)
                                print(Gal(f))
                                print(M)
                                print(disc.factor())
                                print('')